In [22]:
import pandas as pd
import numpy as np

from scipy import stats


In [23]:
alpha = 0.05


In [24]:
observed = [[49,20],[1,30]]
observed

[[49, 20], [1, 30]]

In [25]:
observed = pd.DataFrame(observed)
observed

,0,1
0,49,20
1,1,30


In [26]:
chi2, p, degf, expected = stats.chi2_contingency(observed)

In [27]:
if p < alpha:
    print('We reject the null hypothesis')
else:
    print('We fail to reject the null hypothesis')

We reject the null hypothesis


In [28]:
from pydataset import data

In [29]:
df = data('mpg')
df

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact
...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize
231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize
232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize
233,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize


In [30]:
observed = pd.crosstab(df.year, df.cyl)
observed

cyl,4,5,6,8
year,,,,
1999,45,0,45,27
2008,36,4,34,43


In [31]:
chi2, p, degf, expected = stats.chi2_contingency(observed)

In [32]:
print('Observed')
print(observed.values)
print('\nExpected')
print(expected.astype(int))
print('\n----')
print(f'chi^2 = {chi2:.4f}')
print(f'p     = {p:.4f}')

Observed
[[45  0 45 27]
 [36  4 34 43]]

Expected
[[40  2 39 35]
 [40  2 39 35]]

----
chi^2 = 10.1888
p     = 0.0170


In [52]:
if p < alpha:
    print('We reject the null hypothesis')
else:
    print('We fail to reject the null hypothesis')

We reject the null hypothesis


In [55]:
from env import username, database, password

url = f'mysql+pymysql://{username}:{password}@{database}/employees'


In [57]:
query = ''' 
        select emp_no, gender, dept_name
        from employees
            join dept_emp
                using (emp_no)
            join departments
                using (dept_no)
        where to_date = '9999-01-01'
        and dept_name in ('Marketing','Sales')
        '''

In [60]:
df = pd.read_sql(query, url)
df



,emp_no,gender,dept_name
0,10017,F,Marketing
1,10058,M,Marketing
2,10140,F,Marketing
3,10228,F,Marketing
4,10239,F,Marketing
...,...,...,...
52538,499966,F,Sales
52539,499976,M,Sales
52540,499980,M,Sales
52541,499986,F,Sales


In [62]:
observed = pd.crosstab(df.gender, df.dept_name)
observed

dept_name,Marketing,Sales
gender,,
F,5864,14999
M,8978,22702


In [64]:
chi2, p, degf, expected = stats.chi2_contingency(observed)

In [67]:
query = '''
        select emp_no, gender, dept_no
        from employees
            left join dept_manager
                using (emp_no)
        '''

In [69]:
df = pd.read_sql(query, url)
df

,emp_no,gender,dept_no
0,10001,M,None
1,10002,F,None
2,10003,M,None
3,10004,M,None
4,10005,M,None
...,...,...,...
300019,499995,F,None
300020,499996,M,None
300021,499997,M,None
300022,499998,M,None


In [70]:
df.dept_no.value_counts(dropna=False)

None    300000
d004         4
d006         4
d009         4
d001         2
d002         2
d003         2
d005         2
d007         2
d008         2
Name: dept_no, dtype: int64

In [71]:
df.dept_no = df.dept_no.fillna('not manager')

In [73]:
df.dept_no = np.where( df.dept_no != 'not manager', 
                      'manager', 'not manager')

In [74]:
observed = pd.crosstab(df.gender, df.dept_no)
observed

dept_no,manager,not manager
gender,,
F,13,120038
M,11,179962


In [75]:
chi2, p, degf, expected = stats.chi2_contingency(observed)